---
Paper: https://arxiv.org/pdf/2005.14165

---

---
# Section3: Hyperparameters, AdamW, gradient clipping
---

- Up until now, we have made all the changes in our NN to beter utilize our GPU. Now, we are going to make algorithm changes and improvement of actual optimization itself. 
- And, to do this we like to follow the hyper-parameters that are mentioned in GPT2 or GPT3 paper. There is not much to look in the GPT2 paper or the code that is relesed by GPT2. 
- So, we will look the appendix of GPT3 paper. 
    - B. Details of model training. 

**Change1**
- we change our code as per hyperparameter given in the paper. 

    optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)
    
    will changeed to 

    optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, betas=(0.9, 0.95), eps=1e-8)




**Change2: Gradient Norm Clipping** : 
- we clip the global norm of the gradient at 1.0. 
- This is referening to once we have calculated the gradient after loss.backward()
- we have gradients at all the parameter tensors and what people like to do is basically clip them to have some kind of maximum norm 


norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

- norm: is the norm of the gradient. 
- what this function is doing is calculating the global norm of the parameters. so every single gradient on all the parameters you square it, add it all up and you take a big square root of that and that's the norm of parameter vector basically. It's bascially the length of it. And, we are basically making sure the length of that vector is not more than 1.0. we are going to clip it. 
- And, the reason people like to use is that sometimes you can get unlucky during the optimization may be it's a bad data batch or something like that and if you get very unlucky in a batch you might get really high loss and really high loss can lead to really light gradient and this could basically shock your model and shock the optimizations. 
- So, people like to use gradient norm clipping to prevent the model from getting too big of a shock in term of gradient magnitude and it's upper bounded in this way. 
- It's like hacky solution but people still do it quite frequently. 
- if the value of norm is well behaved then things are good and if it's climbing then things are bad and they are destabilizing during training and sometimes you can get a spike in the norm and means there is some kind of issue or an instability 





In [ ]:
# step 0 | loss: 10.933823585510254 | norm: 29.4195 | dt: 13843.83ms | tok/sec: 591.7439477574981
# step 49 | loss: 5.918039321899414 | norm: 1.2791 | dt: 145.53ms | tok/sec: 56290.43590688744

--- 

**Change3: Learning rate Scheduler**

**Cosine Decay with warmup**


- They don't use a fixed learning rate like we are using up until now. 
- They are using a cosine decay for learning rate 
- And, they way this looks (look at the graph at https://scorrea92.medium.com/cosine-learning-rate-decay-e8b50aa455b)
- Learning rate starts at zero, linearly ramps up for some amount of time and comes down with this cosine sort of form to a minimum lr that is up to you. 
- In paper, they said "we use cosine decay for learning rate down to 10% of its value, over 260 billion tokens (after 260
billion tokens, training continues at 10% of the original learning rate). There is a linear LR warmup over the first 375
million tokens"


